In [2]:
import pandas as pd
import scipy as sp
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from statistics import mean 
from sklearn.model_selection import train_test_split




/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


#### Cross-Domain Learnings
##### According to the research paper using Logistic Regression


In [3]:
df1=pd.read_csv('Drugcom.csv', sep=',')

In [4]:
df1.head()

,Unnamed: 0,Unnamed: 0.1,uniqueID,drugName,condition,review,rating,date,usefulCount,review1,review-list,VAD_neg,VAD_neu,VADpos,VAD_compound,Ratings,Ratings_C,Rating_Model
0,0,0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,"{'neg': 0.136, 'neu': 0.864, 'pos': 0.0, 'comp...","dict_values([0.136, 0.864, 0.0, -0.296])",0.136,0.864,0.000,-0.2960,1,Rating>=7,2
1,1,1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,"{'neg': 0.018, 'neu': 0.892, 'pos': 0.09, 'com...","dict_values([0.018, 0.892, 0.09, 0.8603])",0.018,0.892,0.090,0.8603,1,Rating>=7,2
2,2,2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,"{'neg': 0.041, 'neu': 0.876, 'pos': 0.084, 'co...","dict_values([0.041, 0.876, 0.084, 0.7962])",0.041,0.876,0.084,0.7962,0,4 < Rating <7,1
3,3,3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,"{'neg': 0.027, 'neu': 0.879, 'pos': 0.093, 'co...","dict_values([0.027, 0.879, 0.093, 0.7184])",0.027,0.879,0.093,0.7184,1,Rating>=7,2
4,4,4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,"{'neg': 0.064, 'neu': 0.758, 'pos': 0.178, 'co...","dict_values([0.064, 0.758, 0.178, 0.9403])",0.064,0.758,0.178,0.9403,1,Rating>=7,2


In [5]:
# Boolean Mask with conditions, ISIN, wenn mehrere conditions abgefragt werden

conditions=['Birth Control', 'Depression','Pain', 'Anxiety', 'Diabetes, Type 2']
def model_data(condition):
    df2 =df1[df1['condition']== condition]
    X_train, X_test, y_train, y_test = train_test_split(df2['review'], df2['Rating_Model'], test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test

In [6]:
X_BC_train, X_BC_test, y_BC_train, y_BC_test = model_data('Birth Control')
X_D_train, X_D_test, y_D_train, y_D_test = model_data('Depression')
X_P_train, X_P_test, y_P_train, y_P_test = model_data('Pain')
X_A_train, X_A_test, y_A_train, y_A_test = model_data('Anxiety')
X_Dia_train, X_Dia_test, y_Dia_train, y_Dia_test = model_data('Diabetes, Type 2')


### Creating Inputs 25 Model Variations 

In [7]:
train_str_Inputs = [['X_BC_train', 'y_BC_train'],['X_D_train', 'y_D_train'], ['X_P_train','y_P_train'], ['X_A_train','y_A_train'],['X_Dia_train','y_Dia_train']]
test_str_Inputs = [['X_BC_test','y_BC_test'], ['X_D_test','y_D_test'],['X_P_test','y_P_test'],['X_A_test','y_A_test'],['X_Dia_test','y_Dia_test']]
     
string_combination= [t+j for j in test_str_Inputs for t in train_str_Inputs]


 
train_var_Inputs= [(X_BC_train, y_BC_train),(X_D_train, y_D_train), (X_P_train,y_P_train), (X_A_train,y_A_train),(X_Dia_train,y_Dia_train)]
test_var_Inputs = [(X_BC_test,y_BC_test), (X_D_test,y_D_test),(X_P_test,y_P_test),(X_A_test,y_A_test),(X_Dia_test,y_Dia_test)]
var_combination= [t+j for j in test_var_Inputs for t in train_var_Inputs]




In [8]:
Classifiers = [
            LogisticRegression(),
          
       ]


v = CountVectorizer(analyzer = "word", ngram_range= (1,2))
dicti= {}
dicti['train']=[]
dicti['test']=[]
dicti['accuracy']=[]

for i , (X_train, y_train,X_test,  y_test) in zip(string_combination, var_combination):
        print('-'*100)
        print(i)
        print('-'*100)
        
        
        train_features= v.fit_transform(X_train)

        test_features=v.transform(X_test)
        dense_features=train_features.toarray()
        dense_test= test_features.toarray()


        for classifier in Classifiers:
            try:
                fit = classifier.fit(train_features,y_train)
                pred = fit.predict(test_features)
            except Exception:
                fit = classifier.fit(dense_features,y_train)
                pred = fit.predict(dense_test)
            accuracy = accuracy_score(pred,y_test)
    
            #accuracy = accuracy_score(pred,y_test)
            dicti['train'].append(i[0][2:4])
            dicti['test'].append(i[3][2:4])
            dicti['accuracy'].append(float(accuracy))  
            print('Accuracy of '+classifier.__class__.__name__+'is '+str(accuracy))    




----------------------------------------------------------------------------------------------------
['X_BC_train', 'y_BC_train', 'X_BC_test', 'y_BC_test']
----------------------------------------------------------------------------------------------------
Accuracy of LogisticRegressionis 0.886959267445532
----------------------------------------------------------------------------------------------------
['X_D_train', 'y_D_train', 'X_BC_test', 'y_BC_test']
----------------------------------------------------------------------------------------------------
Accuracy of LogisticRegressionis 0.6404588990632565
----------------------------------------------------------------------------------------------------
['X_P_train', 'y_P_train', 'X_BC_test', 'y_BC_test']
----------------------------------------------------------------------------------------------------
Accuracy of LogisticRegressionis 0.5901484054310072
------------------------------------------------------------------------------

In [36]:
# Procedure for Reducing, Calculation Power
#--> CountVectorizer( min_df-> vocabulary deleted, starting with the lowest )

In [9]:
#How to add to a dictionary
# Update only gives the values which are 


#DataFrame out of Dictionary

cross_domain= pd.DataFrame.from_dict(dicti)
cross_domain['test'].unique()

abbreviation= ['BC', 'D_', 'P_', 'A_', 'Di']

cross_domain.replace(to_replace=abbreviation, value=conditions,inplace=True)
# How to replace all values in a DataFrame


In [10]:
cross_data_logistic=cross_domain.pivot_table(values='accuracy', index= 'train', columns='test')


cross_data_logistic

test,Anxiety,Birth Control,Depression,"Diabetes, Type 2",Pain
train,,,,,
Anxiety,0.877373,0.600147,0.750418,0.688019,0.794872
Birth Control,0.628014,0.886959,0.591380,0.652432,0.659270
Depression,0.810672,0.640459,0.859673,0.708185,0.773176
"Diabetes, Type 2",0.626988,0.610988,0.645172,0.844603,0.548323
Pain,0.785018,0.590148,0.719345,0.666667,0.875247


In [11]:
cross_data_logistic_s= cross_data_logistic.reindex(index=['Depression', 'Anxiety','Pain', 'Birth Control','Diabetes, Type 2' ],columns= ['Depression', 'Anxiety','Pain', 'Birth Control','Diabetes, Type 2' ])

In [13]:
# Calculate the mean overall all columns -> you can change the axis(0-vertical, 1-horizontal)
cross_data_logistic_s['|Train| Avg. accuracy |']=cross_data_logistic_s[cross_data_logistic_s.columns[::-1]].mean(axis=1)
cross_data_logistic_s.loc['|Test| Avg. accuracy |']=cross_data_logistic_s[cross_data_logistic_s.columns[::-1]].mean(axis=0)
cross_data_logistic_s

test,Depression,Anxiety,Pain,Birth Control,"Diabetes, Type 2",|Train| Avg. accuracy |
train,,,,,,
Depression,0.859673,0.810672,0.773176,0.640459,0.708185,0.758433
Anxiety,0.750418,0.877373,0.794872,0.600147,0.688019,0.742166
Pain,0.719345,0.785018,0.875247,0.590148,0.666667,0.727285
Birth Control,0.591380,0.628014,0.659270,0.886959,0.652432,0.683611
"Diabetes, Type 2",0.645172,0.626988,0.548323,0.610988,0.844603,0.655215
|Test| Avg. accuracy |,0.713197,0.745613,0.730178,0.665740,0.711981,0.713342
